In [17]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [44]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [45]:
%run -i ../../zkstats/core.py

In [46]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [47]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))


dummy_data = np.round(np.random.uniform(1,10,len(data)),1)
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.exp(torch.mean(torch.log(dummy_data_tensor)))



In [48]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<=torch.log(torch.tensor(1.01)), self.w)

verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

!@# model= <class '__main__.verifier_model'>
!@# circuit= verifier_model()
!@# circuit.eval= verifier_model()


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_33364/1643889957.py:8: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<=torch.log(torch.tensor(1.01)), self.w)


In [49]:
# prover calculates settings, send to verifier
theory_output = torch.exp(torch.mean(torch.log(data_tensor)))
print("Theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<=torch.log(torch.tensor(1.01)), self.w)


prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, [8], "resources", settings_path)

Theory output:  tensor(47.6981)
!@# model= <class '__main__.prover_model'>
!@# circuit= prover_model()
!@# circuit.eval= prover_model()
==== Generate & Calibrate Setting ====
scale:  [8]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":8,"param_scale":8,"scale_rebase_multiplier":10,"lookup_range":[-6,45056],"logrows":16,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":910,"total_const_size":4,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,8],"model_input_scales":[8],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Ln":{"scale":256.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_33364/21767248.py:11: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<=torch.log(torch.tensor(1.01)), self.w)


In [50]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

spawning module 0


==== setting up ezkl ====


spawning module 2
spawning module 0
spawning module 2


Time setup: 5.308499813079834 seconds
Theory output:  tensor(47.6981)
!@# compiled_model exists? False
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 47.69921875
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[12773529019820117503, 7223165032241168505, 6388811546753741567, 1325363978837834469], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [14148586052410868650, 7343294927740416096, 1182455714804702497, 77845129035718797]]], 'proof': '19dde32cf6a815e0d1e660b28eb75883184b50a49cd026939bed072103ee82080ff44504c9514ca0c048c295d723cdc11dd29e022d169540be8d656025d0916c0ace07b99e446344f44bdb48a38e98bec2e0226c7563598874b4228fa8af7c3409efc3746bd871284e1709d22aa12ad24fb455f7bef8a5c7469f1e7f5b1e2a2f1542516bd8c043fe3b0463af337327f1d3fe38e5ee9ea57f3a6159107ba9b6780d26bcaa57275674783cc90149913ddf940de4f4678329912c8f97f2e0fc05080a83b4f53e9c66de96f35836affe57a29069cc80b7a0647fca02b0c5f18012ce06f1d7e8b12cf4e949ef208a5a775181dd12eb30f30d5b503eb1e0efbd37f65a29d61ebae24b6fa65c9b773a465c3efa64b2a65aed450e321854b879f710eceb00d50b249ef425f1b87c5c17a10f888d170d7f0beef5d602c331add05ade94e427ac672c83211dbdd6b9b0830de2d73b3c4501d2132c89eb232d96dbba10dce12

In [51]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[12773529019820117503, 7223165032241168505, 6388811546753741567, 1325363978837834469], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [14148586052410868650, 7343294927740416096, 1182455714804702497, 77845129035718797]]]
proof boolean:  1.0
proof result 1 : 47.69921875
verified
